In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
%config InlineBackend.figure_format = 'retina'
from qbstyles import mpl_style
mpl_style(dark=True)

In [ ]:
# Load the data from the file
matrix = np.load('correlations.npy')

# cut out the top-left 100x100 section
# matrix = matrix[:100, :100]

# Verify the shape of the matrix
rows, cols = matrix.shape
assert rows == cols

In [ ]:
diag = np.diag(matrix)
self_frequencies = diag
argsorted = np.argsort(self_frequencies)
sort_indices = argsorted[::-1]
sorted_matrix = matrix[sort_indices][:, sort_indices]
sorted_self_frequencies = np.diag(sorted_matrix)

assert np.array_equal(sorted_self_frequencies, diag[sort_indices]), f"first few self-frequencies: {sorted_self_frequencies[:5]}, expected: {diag[sort_indices][:5]}"

print(f"{sort_indices = }")
print(f"diagonal = {np.diag(sorted_matrix)}")

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(sorted_matrix, interpolation="none", cmap='inferno')
plt.colorbar(label="occurences")
plt.title("non-zero co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(sorted_matrix[:256, :256], interpolation='none', cmap='inferno')
plt.colorbar(label="occurences")
plt.title("non-zero co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# scale each co-activation by the self-frequency of the corresponding neuron
scaled_matrix = np.zeros_like(sorted_matrix, dtype=float)
for i in range(rows):
    for j in range(cols):
        scaled_matrix[i, j] = sorted_matrix[i, j] / np.sqrt(
            self_frequencies[i] * self_frequencies[j]
        )

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(scaled_matrix, interpolation="none", cmap='inferno')
plt.colorbar(label="scaled co-activation")
plt.title("scaled co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
BLOCK_SIZE = 4

def cost_function_blocks(order: np.ndarray) -> np.int64:
    """Calculate the cost of a visit order."""
    correlation = np.int64(0)
    for i in range(0, len(order) - BLOCK_SIZE):
        correlation += -np.sqrt(
            matrix[order[i], order[i + 1 : i + BLOCK_SIZE]].sum(dtype=np.int64)
        )
    return correlation

def cost_function_fds(order: np.ndarray) -> float:
    """Calculate the cost of a visit order."""
    # Compute all column differences at once
    col_diffs = matrix[np.ix_(order, order[:-1])] - matrix[np.ix_(order, order[1:])]
    return np.sum(np.sqrt(np.absolute(col_diffs)))

def cost_function_r_criterion(order: np.ndarray) -> float:
    """Calculate the R-criterion cost of a visit order."""
    n = len(order)
    # Create distance matrix (how far each position is from diagonal)
    positions = np.arange(n)
    distances = np.abs(positions[:, None] - positions[None, :])
    
    # Reorder the matrix according to the order
    reordered = matrix[np.ix_(order, order)]
    
    # R-criterion: sum of (distance from diagonal * matrix value)
    return np.sum(distances * reordered)

def cost_function_tsp(order: np.ndarray) -> float:
    """Calculate the TSP-like cost of a visit order."""
    n = len(order)
    total_cost = 0.0
    for i in range(n - 1):
        total_cost += -np.sqrt(matrix[order[i], order[i + 1]])
    return total_cost

def cost_function_absence(order: np.ndarray) -> np.int64:
    """Calculate the cost of a visit order."""
    cost = np.int64(0)
    for i in range(BLOCK_SIZE, len(order) - BLOCK_SIZE):
        cost += matrix[order[i], order[i + BLOCK_SIZE:]].sum(dtype=np.int64)
        cost += matrix[order[i], order[:i - BLOCK_SIZE]].sum(dtype=np.int64)
    return cost

cost_function = cost_function_absence

# heuristic 1: swap two random indices
def swap_two(input: np.ndarray) -> np.ndarray:
    """Swap two random indices."""
    # avoid mutating the original order
    order = input.copy()
    assert not order is input
    a, b = random.sample(range(len(order)), 2)
    order[a], order[b] = order[b], order[a]
    return order

# heuristic 2: reverse a random segment
def reverse_segment(input: np.ndarray) -> np.ndarray:
    """Reverse a random segment."""
    # avoid mutating the original order
    order = input.copy()
    assert not order is input
    a, b = sorted(random.sample(range(len(order)), 2))
    order[a : b + 1] = order[a : b + 1][::-1]
    return order

def simulated_annealing(
    order: np.ndarray,
    initial_temp=1,
    cooling_rate=0.99,
    min_temp=1#e-6
) -> tuple[np.ndarray, np.int64]:
    """Simulated annealing to optimise the visit order."""
    current_order = order[:]
    best_order = order[:]
    current_cost = cost_function(order[:])
    best_cost = current_cost
    temp = initial_temp

    while temp > min_temp:
        new_order = random.choice([swap_two, reverse_segment])(current_order)
        new_cost = cost_function(new_order)

        if (
            new_cost < current_cost
            or np.exp((current_cost - new_cost) / temp) > random.random()
        ):
            current_order, current_cost = new_order.copy(), new_cost
            if new_cost < best_cost:
                best_order, best_cost = new_order.copy(), new_cost
                print(f"New best cost: {best_cost}")

        temp *= cooling_rate

    return best_order, best_cost

In [ ]:
# first, try by starting with the 0..2048 indices
initial_order = np.arange(len(matrix))

# final_best_order_1, final_best_cost_1 = simulated_annealing(initial_order)
final_best_order_2, final_best_cost_2 = simulated_annealing(sort_indices)

print(f"cost of default order  : {cost_function(initial_order)}")
print(f"cost of argsort order  : {cost_function(sort_indices)}")
# print(f"cost of best order 1   : {final_best_cost_1}")
print(f"cost of best order 2   : {final_best_cost_2}")

# print(f"before == after for run 1? {np.array_equal(initial_order, final_best_order_1)}")
eq = np.array_equal(sort_indices, final_best_order_2)
print(f"before == after for run 2? {eq}")
# if not eq:
print(f"before: {sort_indices[:10]}")
print(f"after : {final_best_order_2[:10]}")
print(f"cost before: {cost_function(sort_indices)}")
print(f"cost after : {cost_function(final_best_order_2)}")
# 132643157197

In [ ]:
final_best_order_2 = [295, 674, 622, 514, 382, 164, 483, 558, 421, 959, 393, 103, 187, 359, 167, 408, 557, 330, 306, 559, 841, 76, 988, 869, 489, 704, 880, 63, 415, 325, 753, 507, 586, 132, 339, 395, 836, 346, 69, 157, 353, 814, 946, 928, 475, 151, 183, 15, 72, 106, 717, 871, 723, 632, 244, 68, 271, 1003, 887, 758, 949, 33, 202, 808, 800, 683, 570, 914, 126, 436, 916, 734, 272, 508, 441, 427, 305, 494, 606, 963, 459, 386, 179, 329, 263, 333, 460, 90, 696, 923, 154, 652, 20, 989, 833, 945, 455, 450, 158, 671, 844, 898, 278, 798, 984, 660, 481, 968, 572, 908, 771, 99, 737, 210, 638, 541, 129, 300, 727, 478, 1, 651, 563, 31, 289, 796, 577, 371, 573, 654, 631, 532, 473, 978, 518, 739, 120, 823, 13, 440, 659, 825, 1015, 653, 131, 160, 766, 1021, 285, 173, 830, 192, 561, 338, 958, 801, 600, 533, 287, 456, 490, 77, 750, 380, 854, 961, 828, 149, 80, 299, 92, 443, 843, 81, 281, 485, 745, 953, 83, 680, 397, 511, 493, 604, 847, 629, 26, 971, 218, 608, 601, 48, 23, 461, 926, 221, 53, 685, 972, 692, 406, 1011, 18, 227, 361, 888, 773, 956, 357, 101, 842, 200, 630, 1010, 255, 962, 93, 689, 487, 715, 258, 125, 732, 8, 969, 360, 922, 342, 6, 398, 119, 193, 860, 711, 891, 348, 554, 614, 425, 61, 845, 91, 681, 662, 673, 45, 492, 728, 999, 933, 872, 967, 190, 975, 620, 242, 496, 700, 626, 175, 403, 155, 947, 97, 667, 288, 542, 375, 783, 893, 628, 1023, 970, 940, 501, 519, 199, 744, 591, 576, 621, 663, 141, 992, 321, 332, 0, 718, 110, 995, 517, 676, 568, 55, 34, 684, 59, 185, 497, 708, 656, 642, 666, 902, 345, 135, 925, 89, 875, 930, 223, 901, 248, 226, 832, 789, 466, 113, 993, 40, 216, 390, 509, 180, 594, 416, 584, 691, 482, 759, 32, 553, 705, 118, 212, 105, 981, 1002, 937, 291, 215, 821, 297, 247, 786, 886, 233, 451, 112, 293, 778, 71, 391, 615, 207, 980, 768, 525, 974, 407, 499, 921, 512, 453, 384, 742, 575, 876, 712, 521, 617, 1007, 211, 729, 224, 863, 686, 170, 655, 208, 411, 100, 877, 951, 229, 805, 892, 465, 944, 987, 484, 257, 635, 790, 721, 196, 661, 334, 309, 1000, 249, 290, 820, 835, 468, 352, 219, 740, 308, 534, 765, 480, 301, 813, 286, 29, 910, 270, 605, 351, 469, 795, 942, 664, 153, 567, 565, 966, 54, 840, 161, 495, 564, 43, 418, 649, 50, 372, 220, 145, 672, 957, 449, 811, 195, 181, 862, 409, 907, 237, 915, 388, 124, 488, 720, 36, 310, 938, 636, 163, 611, 303, 539, 562, 911, 827, 446, 206, 240, 668, 936, 424, 432, 115, 616, 274, 401, 498, 57, 787, 505, 410, 647, 598, 623, 364, 452, 165, 665, 799, 770, 319, 236, 979, 924, 899, 392, 130, 943, 146, 794, 66, 279, 552, 920, 16, 755, 358, 302, 625, 741, 307, 433, 592, 607, 896, 350, 528, 927, 362, 37, 423, 434, 774, 389, 917, 191, 873, 1004, 402, 990, 463, 121, 462, 95, 918, 722, 904, 698, 7, 88, 117, 27, 776, 549, 733, 246, 373, 383, 264, 793, 935, 282, 964, 1013, 1017, 900, 1020, 128, 738, 283, 889, 939, 619, 764, 767, 897, 919, 314, 834, 46, 445, 587, 537, 643, 646, 148, 230, 491, 214, 955, 855, 252, 96, 344, 269, 326, 327, 784, 816, 866, 1014, 178, 644, 235, 369, 3, 868, 182, 11, 538, 544, 530, 522, 574, 431, 579, 824, 637, 693, 929, 694, 172, 355, 225, 710, 1018, 560, 354, 142, 954, 555, 109, 448, 640, 79, 201, 313, 748, 30, 516, 578, 648, 912, 239, 426, 529, 217, 817, 85, 510, 114, 294, 387, 867, 595, 197, 280, 328, 545, 47, 464, 536, 861, 618, 829, 347, 413, 527, 476, 479, 950, 730, 420, 123, 706, 864, 262, 540, 639, 803, 703, 67, 931, 996, 228, 535, 581, 682, 261, 754, 51, 815, 998, 994, 337, 735, 960, 2, 137, 702, 454, 515, 98, 627, 251, 87, 677, 471, 589, 213, 188, 839, 241, 150, 266, 429, 894, 697, 785, 714, 260, 39, 222, 439, 531, 932, 317, 812, 82, 719, 985, 253, 209, 613, 879, 588, 602, 777, 374, 205, 140, 254, 168, 49, 982, 657, 609, 292, 467, 580, 503, 1005, 322, 986, 198, 324, 797, 624, 781, 782, 62, 177, 941, 670, 857, 470, 1022, 556, 243, 428, 701, 590, 107, 874, 726, 234, 547, 819, 75, 571, 232, 349, 802, 809, 316, 707, 404, 171, 367, 312, 675, 1016, 381, 486, 273, 679, 336, 41, 997, 903, 340, 585, 250, 147, 238, 757, 447, 851, 882, 1019, 885, 500, 890, 14, 394, 965, 144, 296, 678, 139, 335, 267, 430, 275, 569, 831, 111, 850, 713, 804, 256, 752, 331, 859, 474, 315, 9, 634, 524, 856, 837, 546, 506, 116, 909, 435, 695, 633, 127, 934, 977, 699, 884, 769, 749, 457, 1006, 320, 883, 806, 756, 19, 543, 504, 356, 610, 102, 582, 400, 1012, 848, 379, 599, 268, 788, 323, 669, 810, 983, 1001, 304, 761, 84, 906, 853, 775, 64, 548, 176, 437, 136, 370, 438, 259, 189, 878, 86, 603, 245, 520, 779, 4, 194, 502, 365, 417, 870, 277, 976, 28, 458, 780, 865, 378, 792, 311, 523, 108, 818, 366, 174, 724, 10, 58, 736, 760, 852, 650, 846, 948, 284, 746, 231, 826, 952, 22, 1009, 152, 905, 838, 597, 772, 690, 396, 419, 596, 444, 405, 973, 186, 17, 166, 593, 159, 658, 52, 526, 12, 550, 73, 368, 203, 341, 363, 78, 35, 725, 74, 858, 162, 298, 169, 399, 104, 895, 762, 133, 751, 1008, 134, 44, 709, 24, 807, 25, 513, 991, 731, 641, 318, 412, 56, 566, 385, 881, 65, 184, 94, 791, 376, 422, 716, 688, 583, 143, 763, 551, 612, 377, 70, 122, 743, 442, 477, 204, 849, 414, 747, 156, 38, 42, 60, 687, 21, 5, 472, 822, 645, 276, 913, 343, 138, 265]
# final_best_order_2 = list(range(len(matrix)))

In [ ]:
factored_matrix = matrix[final_best_order_2][:, final_best_order_2]
factored_self_freqs = np.diag(factored_matrix)
scaled_factored_matrix = np.zeros_like(factored_matrix, dtype=float)
for i in range(len(factored_matrix)):
    for j in range(len(factored_matrix)):
        scaled_factored_matrix[i, j] = factored_matrix[i, j] / np.sqrt(
            factored_self_freqs[i] * factored_self_freqs[j]
        )

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(factored_matrix, interpolation="none", cmap='inferno')
plt.colorbar(label="number of co-activations")
plt.title("factored co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(scaled_factored_matrix, interpolation="none", cmap='inferno')
plt.colorbar(label="number of co-activations")
plt.title("factored co-activations")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(scaled_factored_matrix, interpolation="none", cmap='inferno')
plt.axis('off')  # Turn off axes
plt.tight_layout()
plt.show()

In [ ]:
string_0 = ",".join([f"{int(x)}" for x in sort_indices])
# string_1 = ",".join([f"{int(x)}" for x in final_best_order_1])
string_2 = ",".join([f"{int(x)}" for x in final_best_order_2])

# write the orders to a file
with open("visit_order.txt", "w") as f:
    f.write(f"argsort order: {string_0}\n")
    # f.write(f"best order 1 : {string_1}\n")
    f.write(f"best order 2 : {string_2}\n")